LangChain 作为一个框架，由许多包组成。


## langchain-core
此处定义了 LLM、vectorstore、检索器等核心组件的接口。此处未定义第三方集成。有意将依赖项保持为非常轻量级。

## 合作伙伴
虽然集成的长尾部分在 中langchain-community，但我们将流行的集成拆分到它们自己的包中（例如langchain-openai、langchain-anthropic等）。这样做是为了改进对这些重要集成的支持。

## langchain
主langchain包包含构成应用程序认知架构的链、代理和检索策略。这些不是第三方集成。此处的所有链、代理和检索策略都不是特定于任何一种集成的，而是所有集成的通用策略。

# langchain-community
此包包含由 LangChain 社区维护的第三方集成。主要合作伙伴包已分离出来（见下文）。它包含各种组件（LLM、vectorstore、检索器）的所有集成。此包中的所有依赖项都是可选的，以使包尽可能轻量。

## langgraph
langgraph是一种扩展，langchain旨在通过将步骤建模为图中的边和节点，使用 LLM 构建健壮且有状态的多参与者应用程序。

LangGraph 公开了用于创建常见类型代理的高级接口，以及用于编写自定义流程的低级 API。

## langserve
一个将 LangChain 链部署为 REST API 的软件包。可轻松启动和运行可用于生产的 API。

<img src="https://python.langchain.com/v0.2/svg/langchain_stack.svg" width="400" height="200">


## LangChain 表达语言 (LCEL）


LangChain 表达式语言 (LCEL) 是一种声明式的链接 LangChain 组件的方法。LCEL 从第一天开始就被设计为支持将原型投入生产，无需更改代码，从最简单的“prompt + LLM”链到最复杂的链（我们已经看到人们在生产中成功运行了包含 100 多个步骤的 LCEL 链）。以下是您可能想要使用 LCEL 的几个原因：

### 一流的流式支持 
当您使用 LCEL 构建链时，您将获得最佳的第一个令牌时间（直到第一个输出块出现为止所经过的时间）。对于某些链，这意味着例如我们将令牌直接从 LLM 流式传输到流式输出解析器，然后您会以与 LLM 提供程序输出原始令牌相同的速率获得解析后的增量输出块。


### 异步支持 
使用 LCEL 构建的任何链都可以使用同步 API（例如在原型设计时在 Jupyter 笔记本中）以及异步 API（例如在LangServe服务器中）调用。这使得原型和生产中使用相同的代码成为可能，具有出色的性能，并且能够在同一服务器中处理许多并发请求。

### 优化并行执行 
每当您的 LCEL 链具有可以并行执行的步骤时（例如，如果您从多个检索器获取文档），我们都会在同步和异步接口中自动执行此操作，以尽可能减少延迟。

### 重试和回退 
为 LCEL 链的任何部分配置重试和回退。这是让您的链在规模上更可靠的好方法。我们目前正在努力添加对重试/回退的流式支持，这样您就可以获得额外的可靠性而无需任何延迟成本。

### 访问中间结果 
对于更复杂的链，在产生最终输出之前访问中间步骤的结果通常非常有用。这可用于让最终用户知道正在发生的事情，甚至只是调试您的链。您可以流式传输中间结果，并且它在每个LangServe服务器上都可用。

### 输入和输出模式 
输入和输出模式为每个 LCEL 链提供从链结构推断出的 Pydantic 和 JSONSchema 模式。这可用于验证输入和输出，是 LangServe 不可或缺的一部分。

### 无缝 LangSmith 
跟踪 随着您的链条变得越来越复杂，了解每一步究竟发生了什么变得越来越重要。使用 LCEL，所有步骤都会自动记录到LangSmith，以实现最大的可观察性和可调试性。

###无缝 LangServe 部署
 使用 LCEL 创建的任何链都可以使用LangServe轻松部署。



## Runnable 接口
许多 LangChain 组件都实现了该Runnable协议，包括聊天模型、LLM、输出解析器、检索器、提示模板等

这是一个标准接口，可以轻松定义自定义链并以标准方式调用它们。标准接口包括：

stream：流回响应块
invoke：在输入上调用链
batch：在输入列表上调用链

它们还具有相应的异步方法，应与asyncio await语法一起使用以实现并发：

astream：异步流回响应块
ainvoke：在输入异步时调用链
abatch：异步调用输入列表上的链
astream_log：除了最终响应之外，还流回中间步骤的发生
astream_events：链中发生的betalangchain-core流事件（在0.1.14 中引入）


The input type and output type varies by component:

Component	Input Type	Output Type
Prompt	Dictionary	PromptValue
ChatModel	Single string, list of chat messages or a PromptValue	ChatMessage
LLM	Single string, list of chat messages or a PromptValue	String
OutputParser	The output of an LLM or ChatModel	Depends on the parser
Retriever	Single string	List of Documents
Tool	Single string or dictionary, depending on the tool	Depends on the tool

## 组件 Components

LangChain 为各种组件提供标准、可扩展的接口和外部集成，这些组件可用于使用 LLM 进行构建。有些组件是 LangChain 实现的，有些组件我们依赖第三方集成，还有一些则是混合的。

### Chat Models
语言模型使用一系列消息作为输入，并返回聊天消息作为输出（而不是使用纯文本）。这些传统上是较新的模型（较旧的模型通常是LLMs，见上文）。聊天模型支持为对话消息分配不同的角色，帮助区分来自人工智能、用户和系统消息等指令的消息。
尽管底层模型是消息输入、消息输出，但 LangChain 包装器还允许这些模型将字符串作为输入。这意味着您可以轻松地使用聊天模型代替 LLM。

当字符串作为输入传入时，它会转换为 HumanMessage 然后传递给底层模型。

LangChain 不提供任何聊天模型，而是依赖第三方集成。

在构建ChatModel时我们有一些标准化的参数：

model：模型的名称
ChatModels 还接受特定于该集成的其他参数。

## LLMs
语言模型以字符串作为输入并返回字符串。这些传统上是较旧的模型（较新的模型通常是ChatModels，见下文）。
尽管底层模型是字符串输入、字符串输出，但 LangChain 包装器还允许这些模型将消息作为输入。这使得它们可以与 ChatModel 互换。当消息作为输入传入时，它们将在后台格式化为字符串，然后再传递给底层模型。

LangChain 不提供任何 LLM，而是依赖第三方集成。一些语言模型将消息列表作为输入并返回一条消息。有几种不同类型的消息。所有消息都具有role、content和response_metadata属性。

## 提示模板
提示模板有助于将用户输入和参数转换为语言模型的指令。这可用于指导模型的响应，帮助其理解上下文并生成相关且连贯的基于语言的输出。

提示模板以字典作为输入，其中每个键代表提示模板中要填写的变量。

Prompt Templates 输出 PromptValue。此 PromptValue 可以传递给 LLM 或 ChatModel，也可以转换为字符串或消息列表。此 PromptValue 存在的原因是为了便于在字符串和消息之间切换。

有几种不同类型的提示模板

### 字符串
这些提示模板用于格式化单个字符串，通常用于较简单的输入。例如，构造和使用 PromptTemplate 的常见方法如下：

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

value = prompt_template.invoke({"topic": "cats"})
print(type(value))


<class 'langchain_core.prompt_values.StringPromptValue'>


In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant for {topic}"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant for cats'), HumanMessage(content='Tell me a joke about cats')])

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='hi!')])

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("placeholder", "{msgs}") # <-- This is the changed part
])

## 输出解析器
此处的信息指的是解析器从模型中获取文本输出，并尝试将其解析为更结构化的表示。越来越多的模型支持函数（或工具）调用，可自动处理此问题。建议使用函数/工具调用而不是输出解析。

负责获取模型的输出并将其转换为更适合下游任务的格式。当您使用 LLM 生成结构化数据或规范化聊天模型和 LLM 的输出时很有用。

LangChain 有许多不同类型的输出解析器

## 聊天记录
ChatHistory是指 LangChain 中的一个类，可用于包装任意链。这ChatHistory将跟踪底层链的输入和输出，并将它们作为消息附加到消息数据库中。未来的交互将加载这些消息并将它们作为输入的一部分传递到链中。

## 文档加载器
这些类加载Document对象。LangChain 与各种数据源有数百个集成，可从以下数据源加载数据：Slack、Notion、Google Drive 等。

每个 DocumentLoader 都有自己特定的参数，但它们都可以用相同的方法调用.load。示例用例如下：


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    ...  # <-- Integration specific parameters here
)
data = loader.load()

## 文本分割器
加载文档后，您经常需要对它们进行转换，以更好地适应您的应用程序。最简单的例子是，您可能希望将较长的文档拆分为较小的块，以便放入模型的上下文窗口。LangChain 有许多内置的文档转换器，可轻松拆分、合并、过滤和以其他方式操作文档。

当您想要处理长文本时，需要将文本拆分成块。虽然这听起来很简单，但这里却存在很多潜在的复杂性。理想情况下，您希望将语义相关的文本片段放在一起。“语义相关”的含义可能取决于文本的类型。本笔记本展示了几种实现此目的的方法。

从高层次来看，文本分割器的工作原理如下：

将文本分成小的、语义上有意义的块（通常是句子）。
开始将这些小块组合成更大的块，直到达到一定大小（由某些函数衡量）。
一旦达到该大小，就将该块作为其自己的文本，然后开始创建具有一些重叠的新文本块（以保留块之间的上下文）。
这意味着您可以沿着两个不同的轴自定义文本分割器：

文本如何分割
如何测量块大小

## 嵌入模型
Embeddings 类是专为与文本嵌入模型交互而设计的类。有许多嵌入模型提供程序（OpenAI、Cohere、Hugging Face 等） - 此类旨在为所有这些提供程序提供标准接口。

嵌入会创建一段文本的向量表示。这很有用，因为这意味着我们可以在向量空间中思考文本，并执行语义搜索等操作，即在向量空间中寻找最相似的文本片段。

LangChain 中的基础 Embeddings 类提供了两种方法：一种用于嵌入文档，一种用于嵌入查询。前者接受多个文本作为输入，而后者接受单个文本。将它们作为两个独立方法的原因是，某些嵌入提供程序对文档（要搜索的文档）和查询（搜索查询本身）有不同的嵌入方法。



## 向量存储
存储和搜索非结构化数据的最常见方法之一是嵌入数据并存储生成的嵌入向量，然后在查询时嵌入非结构化查询并检索与嵌入查询“最相似”的嵌入向量。向量存储负责存储嵌入数据并为您执行向量搜索。

可以通过执行以下操作将向量存储转换为检索器接口：

vectorstore = MyVectorStore()
retriever = vectorstore.as_retriever()

## 检索器
检索器是一种接口，可根据非结构化查询返回文档。它比向量存储更通用。检索器不需要能够存储文档，只需返回（或检索）文档即可。检索器可以从向量存储中创建，但也足够广泛。检索器接受字符串查询作为输入并返回文档列表作为输出。

## 工具
工具是代理、链或聊天模型/LLM 可以用来与世界互动的接口。
工具由以下组件组成：

工具名称
该工具的功能描述
该工具输入的 JSON 模式
要调用的函数
工具的结果是否应直接返回给用户（仅与代理相关）

名称、描述和 JSON 模式作为 LLM 的上下文提供，从而允许 LLM 确定如何适当地使用该工具。

给定可用工具列表和提示，LLM 可以请求使用适当的参数调用一个或多个工具。

一般来说，在设计聊天模型或 LLM 使用的工具时，需要牢记以下几点：

针对工具调用进行过微调的聊天模型在工具调用方面会比未经微调的模型表现得更好。
未经微调的模型可能根本无法使用工具，尤其是在工具很复杂或需要多次工具调用的情况下。
如果工具具有精心选择的名称、描述和 JSON 模式，模型的性能将更好。
与复杂的工具相比，简单的工具通常更容易被模型使用。

## 工具包
是一组工具的集合，旨在一起用于特定任务。它们具有方便的加载方法。

所有工具包都公开一个get_tools返回工具列表的方法。因此您可以执行以下操作：

# Initialize a toolkit
toolkit = ExampleTookit(...)

# Get list of tools
tools = toolkit.get_tools()